# Geodesics of Poincare half-plane

In a chart $(U,\phi)$ the curve $c:[a,b]\to M$ has the coordinates $y(t):=(\phi\circ c)(t) = (x^1(c(t)),\dots,x^n(c(t)))$ and $y^i(t)=(x^i\circ c)(t)$. With $\dot{y}^i:=\frac{dy^i}{dt}$ the covariant derivative of the velocity field is
\begin{align*}
T(t) = c^\prime(t) = \dot{y}^i\partial_{i,c(t)} && \frac{DT}{dt}(t)=\ddot{y}^i\partial_i + \dot{y}^i\nabla_{c^\prime(t)}\partial_i =\ddot{y}^i\partial_i + \dot{y}^i\nabla_{\dot{y}^j\partial_{j_1},c(t)}\partial_i=(\ddot{y}^k+\dot{y}^i\dot{y}^j\Gamma_{ij}^k)\partial_k
\end{align*} 
Thus, $c(t)$ is a geodesic at $p\in M$ with $c'(0)=X\in T_pM$ if the system of (nonlinear) differential equations are fulfilled
\begin{align}
\ddot{y}^k+\dot{y}^i\dot{y}^j\Gamma_{ij}^k=0,\qquad k=1,\dots,n.
\end{align}

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# define right-hand side (s... time, v=[gamma^1,gamma^2,v^1,v^2])
def rhs(s, v): 
    # compute Christoffel symbols of second kind
    chr111 = 0
    chr121 = -1/v[1]
    chr221 = 0
    chr112 = 1/v[1]
    chr122 = 0
    chr222 = -1/v[1]
    return [v[2],v[3],-v[2]**2*chr111-2*v[2]*v[3]*chr121-v[3]**2*chr221,-v[2]**2*chr112-2*v[2]*v[3]*chr122-v[3]**2*chr222]


Tend = 0.2
refval = solve_ivp(rhs, (0, Tend), [0.8,0.1,0.4,1],t_eval=np.linspace(0, Tend,1000))
plt.plot(refval.y.T[:,0],refval.y.T[:,1])
plt.axis('equal')
plt.show()

In [ ]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw
import numpy as np
import matplotlib.pyplot as plt
from time import time

In [ ]:
Gex = 1/y**2*CF( (1,0,0,1),dims=(2,2) )

# Check if the segment [A,B] intersects with segment [C,D]
# returnts true/false and the intersection point if they intersect
def segment_intersect(A, B, C, D):
    # If the segments are parallel, they do not intersect, then check if points are on the same side
    if np.abs(np.cross(B-A, D-C)) < 1e-12: return (False,None)
    if np.dot(np.cross(B-A,C-A),np.cross(B-A,D-A))>0: return (False,None)
    if np.dot(np.cross(D-C,A-C),np.cross(D-C,B-C))<=0:
        s = (np.cross(A,B-A)-np.cross(C,B-A))/np.cross(D-C,B-A)
        return (True, C+s*(D-C))
    else:
        return (False,None)

# Check if segment [P,Q] intersects with edges of triangle
# If it intersects with two edges, increase P in direction Q to obtain a single intersection
# returns intersection point and points of corresponding edge
def segm_trig_intersect(P,Q,el,mesh):
    A = np.array(mesh[el.vertices[0]].point)
    B = np.array(mesh[el.vertices[1]].point)
    C = np.array(mesh[el.vertices[2]].point)
    res1,res2,res3=True,True,True
    scale = 1e-7
    diff = Q-P
    while int(res1+res2+res3)>1:
        newP = P + scale*diff
        res1,P1 = segment_intersect(A,B, newP, Q)
        res2,P2 = segment_intersect(A,C, newP, Q)
        res3,P3 = segment_intersect(C,B, newP, Q)
        scale *=2
    if res1: return (P1, A, B)
    if res2: return (P2, A, C)
    if res3: return (P3, C, B)
    else: print("Warning: segement does not intersect trig")
        
# Compute the kinetic energy of the geodesic
# q.. position, p.. speed of geodesic
def KineticEnergy(q,p,G,mesh):
    G_eval = np.array(G(mesh(*q))).reshape(2,2)
    return 0.5*G_eval.dot(p).dot(p)

# Finds new element if geodesic crosses at least one interface.
# Returns the closest point of the old element to the intersection point 
# as well as the new element and closest point from the new element to the intersection point
# and how much the time step dt has to be scaled to reach the interface
def BisectElements(oldpoint,currentpoint,direction,old_el,mesh,dt):
    tmp_el = mesh[ElementId(mesh(*currentpoint).nr)]
    a = 0
    b = 1
    other_el = tmp_el
    near_oldp = oldpoint
    near_newp = oldpoint + 1*dt*direction
    while (b-a)>1e-6: # as long as tolerance is not reached
        s = (b+a)/2
        tmppnt = oldpoint + s*dt*direction
        tmp_el = mesh[ElementId(mesh(*tmppnt).nr)]
        if tmp_el.nr == old_el.nr:
            near_oldp = tmppnt
            a = s
        else:
            near_newp = tmppnt
            other_el = tmp_el
            b = s
    return (near_oldp, near_newp, other_el,s)

In [ ]:
# right-hand side of ODE
def rhs(s, v, mesh, chr2):        
    try:
        chr_eval = chr2(mesh(*v[:2]))
    except:
        return 0*v 
    chr111 = chr_eval[4*0+2*0+0]
    chr121 = chr_eval[4*0+2*1+0]
    chr221 = chr_eval[4*1+2*1+0]
    chr112 = chr_eval[4*0+2*0+1]
    chr122 = chr_eval[4*0+2*1+1]
    chr222 = chr_eval[4*1+2*1+1]
    return [v[2],v[3],-v[2]**2*chr111-2*v[2]*v[3]*chr121-v[3]**2*chr221,-v[2]**2*chr112-2*v[2]*v[3]*chr122-v[3]**2*chr222]

# Solve ODE with initial values (x0,y0) initial position, (v0,v1) initial velocity
# mesh, Christoffel symbol of second kind, time step dt and end time
def SolveODE(x0,y0,v0,v1, mesh, chr2, dt, Tend=1):
    T=0
    numsteps = int(Tend/dt)
    values = [np.array([x0,y0,v0,v1])]
    
    currentpoint = values[0][:2]
    oldpoint = currentpoint
    old_el = None
    el = mesh[ElementId(mesh(*values[0][:2]).nr)]
    ts = [T]
    while T < Tend:
        # explicit Euler step
        inc = np.array(rhs(T,values[-1][:], mesh, chr2))
        values.append(values[-1][:] + dt*inc)
        
        oldpoint = currentpoint
        currentpoint = values[-1][:2]
        old_el = el
        
        # new point out of mesh ?
        if mesh(*values[-1][:2]).nr < 0: return values[:-1][:]
        
        el = mesh[ElementId(mesh(*values[-1][:2]).nr)]
        
        # Geodesic crosses face
        if old_el.nr != el.nr:
            intersection_pnt,V1,V2 = segm_trig_intersect(oldpoint,currentpoint,old_el,mesh)
            
            near_oldp, near_newp, el,s = BisectElements(oldpoint,currentpoint,values[-2][2:],old_el,mesh,dt)
            T += s*dt # Update time to hit the interface
            
            values[-1][:2] = near_newp
            intersection_pnt_old = near_oldp
            intersection_pnt_new = near_newp
            
            gold = np.array(gfG(mesh(*intersection_pnt_old))).reshape(2,2)
            gnew = np.array(gfG(mesh(*intersection_pnt_new))).reshape(2,2)

            # Euclidean tangent and normal vector
            t = 1/np.linalg.norm(V2-V1)*(V2-V1)
            n = np.array( [t[1],-t[0]])
            
        else:
            T += dt
        ts.append(T)
        
    return values, ts

results = []
maxhs=[0.05,0.025,0.0125,0.0125/2]

Tend = 0.2
init = [0.8,0.1,0.4,1]
          
energy = []
ts = []
with TaskManager():
    for maxh in maxhs:
        shape = MoveTo(0.6,0.05).Rectangle(0.7,0.5).Face()
        mesh = Mesh(OCCGeometry(shape,dim=2).GenerateMesh(maxh=maxh))
    
        gfG = GridFunction(HCurlCurl(mesh,order=0))
        gfG.Set(Gex, dual=True)
        chr2 = gfG.Operator("christoffel2")
        values, t = SolveODE(x0=init[0],y0=init[1],v0=init[2],v1=init[3], \
                                    mesh=mesh, chr2=chr2, dt=0.001, Tend=Tend)
        results.append(np.array(values))
        energy.append([])
        ts.append(t)
        for k in range(len(results[-1][:,0])):
            energy[-1].append(KineticEnergy(results[-1][k,:2],results[-1][k,2:],mesh=mesh,G=gfG))

for i in range(len(results)):
    plt.plot(results[i][:,0],results[i][:,1], label="h"+str(maxhs[i]))
plt.plot(refval.y.T[:,0],refval.y.T[:,1], '-' , color="k", label="reference")
plt.legend()
plt.axis('equal')
plt.ticklabel_format(useOffset=False)
plt.show()


# plot energy here
for i in range(len(energy)):
    plt.plot(ts[i],energy[i])
plt.ticklabel_format(useOffset=False)
plt.show()   

# lines for plotting on mesh
lpts = []
for i in range(len(results)):
    lpts.append([])
    for k in range(len(results[i][:,0])-1):
        lpts[-1] += [results[i][k,0], results[i][k,1], 0.0, \
            results[i][k+1,0], results[i][k+1,1], 0.0]
            
lines = []
colors=["red","blue","magenta","cyan","orange", "teal", "brown", "white","red"]
for i in range(len(lpts)):
    lines.append({ "type": "lines", "position": lpts[i], "name": "my lines", "color" : colors[i]})

Draw (mesh,objects=lines);